# BERT Weather Condition Custom Training and Sentiment Analysis Evaluation

Import the necessary libraries and packages. Pandas for the dataframe and in-memory data management. Numpy for linear algebra, wordcloud and matplotlib for visualization.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from wordcloud import WordCloud
import os

Import and read the data into a pandas dataframe

In [3]:
path = "daily_weather_2020.csv" # path to data csv
if(os.path.exists(path)):
    print("File Found")
    df = pd.read_csv(path, usecols=['summary', 'icon'], low_memory=True, dtype=str)
else:
    print("no file found")

File Found


# Data Cleaning and Validation setup

Replace the icon column of the dataframe and check all the data was loaded properly as expected. The icon column items are replaced with the proper sentiment instead of the current icon value. This will help with validation when training the data. The entire dataset has validation.

In [4]:
# Print the dataframe basic information
df = df.rename(columns={'summary':'statement', 'icon': 'sentiment'})
df.info()
df.describe()
# Set up the validation column in the dataframe without modifying the raw data
def setValidation():
    print("SETTING UP DATA VALIDATION")
    for i in range(len(df.sentiment)):
        val = df.sentiment[i]
        if val == 'rain' or val == 'snow' or val == 'wind' or val == 'fog':
            df.sentiment[i] = 'unfavorable' # negative sentiment
        elif val == 'clear-day':
            df.sentiment[i] = 'favorable' # positive sentiment
        elif val == 'partly-cloudy-day' or val == 'cloudy':
            df.sentiment[i] = 'neutral' # neutral (possibly not enough data)
    print("VALIDATION HAS BEEN SET UP")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30688 entries, 0 to 30687
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   statement  30685 non-null  object
 1   sentiment  30688 non-null  object
dtypes: object(2)
memory usage: 479.6+ KB


The validation that is setup can be tested by running the snippet below. This is a time consuming and resource intensive step, so it is not recommended to run this when reusing the same dataset.

In [5]:
# check the validation edit worked
def checkValidationSetting():
    print("CHECKING VALIDATION DATA")
    favorable_count = 0
    unfavorable_count = 0
    neutral_count = 0

    for i in range(len(df.sentiment)):
        if(df.sentiment[i] == 'favorable'):
            favorable_count +=1
        elif(df.sentiment[i] == 'unfavorable'):
            unfavorable_count +=1
        elif(df.sentiment[i] == 'neutral'):
            neutral_count +=1

    print('Favorable Count: ' + str(favorable_count))
    print('Unfavorable Count: ' + str(unfavorable_count))
    print('Neutral Count: ' + str(neutral_count))


setValidation()
checkValidationSetting()
print('PROCESS COMPLETED')

SETTING UP DATA VALIDATION
VALIDATION HAS BEEN SET UP
CHECKING VALIDATION DATA
Favorable Count: 8527
Unfavorable Count: 12904
Neutral Count: 9257
PROCESS COMPLETED


# Test and Train DataFrames

In [6]:
train,eva = train_test_split(df,test_size=0.2)

# Model Setup

The BERT model is setup here with the sentiment classifiers. The model segments into three classes by default. These three classes are mapped to condition sentiments in the model setup.

In [7]:
!pip install simpletransformers

  Using cached simpletransformers-0.63.4-py3-none-any.whl (248 kB)
  Using cached sentencepiece-0.1.96-cp36-cp36m-macosx_10_6_x86_64.whl (1.2 MB)
  Using cached streamlit-1.5.1-py2.py3-none-any.whl (9.7 MB)
  Using cached seqeval-1.2.2-py3-none-any.whl
  Using cached tokenizers-0.11.5.tar.gz (220 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached regex-2022.1.18-cp36-cp36m-macosx_10_9_x86_64.whl (289 kB)
  Using cached datasets-1.18.3-py3-none-any.whl (311 kB)
  Using cached transformers-4.16.2-py3-none-any.whl (3.5 MB)
  Using cached tqdm-4.62.3-py2.py3-none-any.whl (76 kB)
  Using cached wandb-0.12.10-py2.py3-none-any.whl (1.7 MB)
  Using cached sacremoses-0.0.47-py2.py3-none-any.whl (895 kB)
  Using cached PyYAML-6.0-cp36-cp36m-macosx_10_9_x86_64.whl (189 kB)
  Using cached huggingface_hub-0.4.0-py3-none-any.whl (67 kB)
  Using cached filelock-3.4.1-py3-none-any.whl (9.9 kB)
  Usin

In [8]:
from simpletransformers.classification import ClassificationModel

# Create a TransformerModel
model = ClassificationModel('bert', 'bert-base-cased', num_labels=3, args={'reprocess_input_data': True, 'overwrite_output_dir': True},use_cuda=True)

ModuleNotFoundError: No module named 'simpletransformers'

The default model outputs are overwritten here and replaced with the relevant condition sentiment

In [ ]:
# 0,1,2 : positive,negative
def making_label(st):
    if(st=='positive'):
        return 0
    elif(st=='neutral'):
        return 2
    else:
        return 1

train['label'] = train['sentiment'].apply(making_label)
eva['label'] = eva['sentiment'].apply(making_label)
print(train.shape)

The dataset is prepared for training via the creation of a test DataFrame and Evaluation DataFrame

In [ ]:
train_df = pd.DataFrame({
    'text': train['statement'][:1500].replace(r'\n', ' ', regex=True),
    'label': train['label'][:1500]
})

eval_df = pd.DataFrame({
    'text': eva['statement'][-400:].replace(r'\n', ' ', regex=True),
    'label': eva['label'][-400:]
})

The DataFrames are then run through testing and new result variables are created for evaluating model metrics

In [ ]:
model.train_model(train_df)
result, model_outputs, wrong_predictions = model.eval_model(eval_df)

# Model Metrics and Model Performance

SciKit is used to validate the model and test how the model performs when trained with this dataset. Anything above 90% is enough for the model to be considered useful and accurate.

In [ ]:
print(result)
print(model_outputs)

lst = []
for arr in model_outputs:
    lst.append(np.argmax(arr))

true = eval_df['label'].tolist()
predicted = lst

In [ ]:
import sklearn
mat = sklearn.metrics.confusion_matrix(true , predicted)
print(mat)

df_cm = pd.DataFrame(mat, range(3), range(3))

sns.heatmap(df_cm, annot=True)
plt.show()

sklearn.metrics.classification_report(true,predicted,target_names=['favorable','neutral','unfavorable'])
print('Model Accuracy: ', 100*sklearn.metrics.accuracy_score(true,predicted), '%')

# Live Demo and Model Testing
This function allows for the model to be evaluated with manually prompted data, a user interface for evaluating the model with real time and human generated queries.

In [ ]:
def get_result(statement):
    result = model.predict([statement])
    pos = np.where(result[1][0] == np.amax(result[1][0]))
    pos = int(pos[0])
    sentiment_dict = {0:'positive',1:'negative',2:'neutral'}
    print(sentiment_dict[pos])
    return

sentiment = get_result(input("Input a phrase for Validation: "))
print("The input data was classified as:", sentiment)